# REF Assessment Notebook

### 31st May 2021

### Neil D. Lawrence

This notebook is a helper for reviewing ref papers.


In [ ]:
import ref

In [ ]:
data = ref.assess.data()

In [ ]:
write_data = ref.access.upload()

In [ ]:
INDEX = 207591

In [ ]:
ref.assess.score(INDEX, data, write_data)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 5))
write_data.hist(column='Score', grid=False, bins=np.linspace(-0.5, 12.5, 14), width=0.8, ax=ax)
_ = ax.set_xticks(np.linspace(0,12,13))

In [ ]:
write_data['Score'].count()